In [ ]:
from ibapi.account_summary_tags import AccountSummaryTags
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from datetime import datetime
from math import isnan

import threading
import time
import pandas
from pandas import DataFrame

In [ ]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []  # Initialize variable to store candle
        self.nextorderId = None

    def historicalData(self, reqId, bar):
        print(f'reqId: {reqId} Time: {bar.date} Close: {bar.close}')
        self.data.append([reqId, bar.date, bar.close])

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId,
                    whyHeld, mktCapPrice):
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining,
              'lastFillPrice', lastFillPrice)

    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action,
              order.orderType, order.totalQuantity, orderState.status)

    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId,
              execution.orderId, execution.shares, execution.lastLiquidity)

    # def updateAccountValue(self, key:str, val:str, currency:str,
    #                         accountName:str):

    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        super().accountSummary(reqId, account, tag, value, currency)
        print("AccountSummary. ReqId:", reqId, "Account:", account, "Tag: ", tag, "Value:", value, "Currency:",
              currency)

    def accountSummaryEnd(self, reqId: int):
        super().accountSummaryEnd(reqId)
        print("AccountSummaryEnd. ReqId:", reqId)


In [ ]:
app = IBapi()
app.connect('127.0.0.1', 7497, 10645)


def run_loop():
    app.run()

In [ ]:
reqId_serial = 1

In [ ]:
# Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(1)  # Sleep interval to allow time for connection to server

In [ ]:
def create_contract(curr1, curr2):
    # Create contract object
    contract = Contract()
    contract.symbol = curr1
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = curr2
    return contract

In [ ]:
EUR_USD_contract = create_contract('EUR', 'USD')

In [ ]:
# Request historical candles
app.reqHistoricalData(reqId=1, contract=EUR_USD_contract, endDateTime='', durationStr='1 D', barSizeSetting='1 day',
                      whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
time.sleep(2)  # sleep to allow enough time for data to be returned

In [ ]:
df = pandas.DataFrame(app.data, columns=['reqId', 'DateTime', 'Close'])

In [ ]:
df['DateTime'] = [datetime.strptime(dt, '%Y%m%d') for dt in df['DateTime']]

In [ ]:
df

In [ ]:
def request_data(contract):
    global reqId_serial
    reqId_serial += 1
    app.reqHistoricalData(reqId=reqId_serial, contract=contract, endDateTime='', durationStr='1 D',
                          barSizeSetting='1 day',
                          whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
    time.sleep(0.5)
    if app.data[len(app.data) - 1][0] != reqId_serial:
        return float('NaN')
    else:
        return app.data[len(app.data) - 1][2]

In [ ]:
print(request_data(EUR_USD_contract))

In [ ]:
def fetch_exc_rate(base, quote):
    contract = create_contract(base, quote)
    return request_data(contract)

In [ ]:
currencies = ['USD', 'EUR', 'GBP', 'JPY', 'AUD']

In [ ]:
def fetch_all(curr_list):
    matrix = DataFrame(columns=curr_list, index=curr_list)
    for i in range(0, len(curr_list)):
        for j in range(0, len(curr_list)):
            if i == j:
                matrix[curr_list[i]][curr_list[j]] = 1
            else:
                matrix[curr_list[i]][curr_list[j]] = fetch_exc_rate(curr_list[i], curr_list[j])
    return matrix

In [ ]:
exchange_rates = fetch_all(currencies)

In [ ]:
def fill_in_nan(matrix):
    for col in matrix.columns:
        for row in matrix.index:
            if isnan(matrix[col][row]) & (not isnan(matrix[row][col])):
                matrix[col][row] = 1 / matrix[row][col]
    return matrix

In [ ]:
def check_all_data(matrix):
    return not matrix.isnull().values.any()


In [ ]:
exchange_rates = fill_in_nan(exchange_rates)

In [ ]:
check_all_data(exchange_rates)

In [ ]:
exchange_rates

In [ ]:
def check_arbitrage(result_dict, matrix, curr1, curr2, curr3):
    arbitrage_amount = matrix[curr1][curr2] * matrix[curr2][curr3] * matrix[curr3][curr1] - 1
    result_dict[curr1 + '.' + curr2 + '.' + curr3] = arbitrage_amount
    arbitrage_amount_rev = matrix[curr1][curr3] * matrix[curr3][curr2] * matrix[curr2][curr1] - 1
    result_dict[curr1 + '.' + curr3 + '.' + curr2] = arbitrage_amount_rev
    print(curr1 + '->' + curr2 + '->' + curr3 + '->' + curr1 + ': ' + "{:.6f}".format(arbitrage_amount))
    print(curr1 + '->' + curr3 + '->' + curr2 + '->' + curr1 + ': ' + "{:.6f}".format(arbitrage_amount_rev))
    return arbitrage_amount

In [ ]:
def check_all_arbitrage(result_dict, matrix, currency_list):
    for i in range(0, len(currency_list)):
        for j in range(i + 1, len(currency_list)):
            for k in range(j + 1, len(currency_list)):
                check_arbitrage(result_dict, matrix, currency_list[i], currency_list[j], currency_list[k])
    return result_dict

In [ ]:
results = dict()
results = check_all_arbitrage(results, exchange_rates, currencies)

In [ ]:
max_path = max(results, key=results.get)

In [ ]:
def FX_order_contract(curr1, curr2):
    contract = Contract()
    contract.symbol = curr1
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = curr2
    return contract


In [ ]:
def create_order(quantity, action='BUY', orderType='MKT', lmtPrice=None):
    order = Order()
    order.action = action
    order.totalQuantity = quantity
    order.orderType = orderType
    if orderType == 'LMT' and lmtPrice is not None:
        order.lmtPrice = lmtPrice
    return order

In [ ]:
usd_eur_order = create_order(100000, 'BUY', 'MKT')
usd_eur_contract = FX_order_contract('USD', 'EUR')
app.placeOrder(app.nextValidId, usd_eur_contract, usd_eur_order)

In [ ]:
order = Order
order.action = 'BUY'
order.totalQuantity = 100000
order.orderType = 'MKT'
order.lmtPrice = 1.10

app.placeOrder(app.nextValidId, usd_eur_contract, order)

In [ ]:
app.reqAccountSummary(2, 'All', AccountSummaryTags.AllTags)
time.sleep(1)

In [13]:
app.reqAccountSummary(2, 'All', AccountSummaryTags.AllTags)
time.sleep(1)

AccountSummary. ReqId: 2 Account: DU5016513 Tag:  AccountType Value: INDIVIDUAL Currency: 
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  Cushion Value: 1 Currency: 
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  DayTradesRemaining Value: -1 Currency: 
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  LookAheadNextChange Value: 0 Currency: 
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  AccruedCash Value: 0.00 Currency: USD
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  AvailableFunds Value: 1000000.00 Currency: USD
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  BuyingPower Value: 4000000.00 Currency: USD
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  EquityWithLoanValue Value: 1000000.00 Currency: USD
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  ExcessLiquidity Value: 1000000.00 Currency: USD
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  FullAvailableFunds Value: 1000000.00 Currency: USD
AccountSummary. ReqId: 2 Account: DU5016513 Tag:  FullExcessLiquidity V